# Tarea 2

Integrantes:
* Juan Maulén 
* Pedro Pérez
* Tomás Rojas

In [ ]:
from torchvision.datasets import DatasetFolder
import torch
import random
from PIL import Image
import numpy as np
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, SubsetRandomSampler, RandomSampler

In [ ]:
def random_rotation(im, theta):
    theta = random.uniform(*theta)
    return im.rotate(theta)

def random_flip(im):
    if bool(random.getrandbits(1)):
        im.transpose(Image.FLIP_LEFT_RIGHT)
    return im
    
def normalize_im(im):
    im = np.asarray(im)
    
def loader(path):
    transformation = ToTensor()
    
    im = Image.open(path).resize((224, 224))  # escalamiento de la imagen
    im = random_flip(im)  # reflexion de la imagen
    im = random_rotation(im, (-20, 20))  # rotacion aleatoria de la imagen
    
    
    tensor = transformation(im)  # se transforma la imagen a un tensor
    if tensor.shape[0] == 1:
        tensor = tensor.repeat(3, 1, 1)
    
    rand_tensor = (1.5 - 1.2) * torch.rand((3, 224, 224)) + 1.2
    return tensor * rand_tensor


In [ ]:
train_folder = DatasetFolder('./chest_xray/train', loader=loader, extensions='.jpeg')
# distribución real
test_folder = DatasetFolder('./chest_xray/test', loader=loader, extensions='.jpeg')

In [ ]:
def histogram(train_folder, test_folder):
    train_folder_x, train_folder_y = np.unique(train_folder.targets, return_counts=True)
    test_folder_x, test_folder_y = np.unique(test_folder.targets, return_counts=True)
    plt.figure(figsize=(10,5))
    plt.bar(["'NORMAL' train","'PNEUMONIA' train"], 
            train_folder_y)
    plt.bar(["'NORMAL' test","'PNEUMONIA' test"], 
            test_folder_y)

    p_normal_train = train_folder_y[0] / train_folder_y.sum()
    p_pneumonia_train = train_folder_y[1] / train_folder_y.sum()
    p_normal_test = test_folder_y[0] / test_folder_y.sum()
    p_pneumonia_test = test_folder_y[1] / test_folder_y.sum()


    plt.text(-.1, train_folder_y[0] + 15,
             '{:.2f}%'.format(p_normal_train * 100))

    plt.text(-.1 + 1, train_folder_y[1] + 15,
             '{:.2f}%'.format(p_pneumonia_train * 100))

    plt.text(-.1 + 2, test_folder_y[0] + 15,
             '{:.2f}%'.format(p_normal_test * 100))

    plt.text(-.1 + 3, test_folder_y[1] + 15,
             '{:.2f}%'.format(p_pneumonia_test * 100))
    return

In [ ]:
histogram(train_folder, test_folder)

In [ ]:
# train test split
n_test =  int(len(train_folder) * .2)  
n_train = len(train_folder) - n_test
conjunto_train, conjunto_val = torch.utils.data.random_split(train_folder, [n_train, n_test])

In [ ]:
class ReplicarMuestreoDePrueba(torch.utils.data.Sampler):
    
    def __init__(self, etiquetas_prueba, indices_val, etiquetas_val):
        if len(etiquetas_val) != len(indices_val):
            etiquetas_val = np.array(etiquetas_val)[indices_val]
        self.etiquetas_prueba = np.array(etiquetas_prueba)
        self.indices_val = np.array(indices_val)
        self.etiquetas_val = np.array(etiquetas_val)
        
    def __iter__(self):
        # {'NORMAL': 0, 'PNEUMONIA': 1}
        clases_prueba, frecuencias_prueba = np.unique(self.etiquetas_prueba, 
                                                      return_counts=True)
        
        p_normal_deseada = frecuencias_prueba[0] / frecuencias_prueba.sum()
        
        indices_val_0 = self.indices_val[np.where(np.array(self.etiquetas_val)==0)]
        indices_val_1 = self.indices_val[np.where(np.array(self.etiquetas_val)==1)]
        
        n_0 = len(indices_val_0)
        n_1 = int(n_0 / p_normal_deseada - n_0)
        
        indices_val_1 = np.random.choice(indices_val_1, n_1)
        
        lista_con_indices = indices_val_0.tolist() + indices_val_1.tolist()
        return iter(lista_con_indices)

In [ ]:
# train_indices
# train_targets
# test_indices
# test_targets
# son sub conjuntos de data_train

train_indices, train_targets = conjunto_val.indices, conjunto_val.dataset.targets
test_indices, test_targets = conjunto_train.indices, conjunto_train.dataset.targets

In [ ]:
test_sampler = ReplicarMuestreoDePrueba(test_folder.targets, 
                                        conjunto_val.indices, 
                                        conjunto_val.dataset.targets)

training_set = SubsetRandomSampler(test_sampler)
test_set = RandomSampler(conjunto_val)
validation_set = DataLoader(conjunto_val)

In [1]:
# P2

In [2]:
import torch

torch.manual_seed(4020)

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class DWSepConv2d(nn.Module):
    
    def __init__(self,  in_channels, out_channels, kernel_size, padding, bias=True):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size, padding, bias = bias)
        self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias = bias)
        
    def forward(self, x):
        xb = self.conv1(x)
        xb = F.relu(self.conv2(xb))
        return xb

In [4]:
class VGG16DWSep(torch.nn.Module):
    
    def __init__(self):
        super()._init_()
        self.conv1layer1=nn.Conv2d(3,64,kernel_size=3,padding=1)
        self.conv1layer2=nn.Conv2d(64,64,kernel_size=3,padding=1)
        self.maxpool2d1=nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv2layer1=DWSepConv2d(64,128,3,1)
        self.conv2layer2=DWSepConv2d(128,128,3,1)
        self.maxpool2d2=nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv1=DWSepConv2d(128,256,3,1)
        self.batchnorm1=nn.BatchNorm2d(256)
        
        self.conv2=DWSepConv2d(256,256,3,1)
        self.batchnorm2=nn.BatchNorm2d(256)
        
        self.conv3=DWSepConv2d(256,512,3,1)
        self.maxpool2d3=nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv4=DWSepConv2d(512,512,3,1)
        self.batchnorm4=nn.BatchNorm2d(512)
        
        self.conv5=DWSepConv2d(512,512,3,1)
        self.batchnorm5=nn.BatchNorm2d(512)
        
        self.conv6=DWSepConv2d(512,512,3,1)
        self.maxpool2d6=nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.flatten=nn.Flatten()
        
        self.fc1 = nn.Linear(256, 1024)
        self.dropout1=nn.Dropout(.7)
        
        self.fc2 = nn.Linear(1024, 512)
        self.dropout2=nn.Dropout(.5)
        
        self.fc3 = nn.Linear(512, 2)
    
       
    def forward():
        xb = xb.view(-1, 3, 224, 224)
        xb = F.relu(self.conv1layer1(xb.float()))
        xb = F.relu(self.conv1layer2(xb))
        xb = F.relu(self.maxpool2d1(xb))
        
        xb = F.relu(self.conv2layer1(xb))
        xb = F.relu(self.conv2layer2(xb))
        xb = F.relu(self.maxpool2d2(xb))
        
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.batchnorm1(xb))
        
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.batchnorm2(xb))
        
        xb = F.relu(self.conv3(xb))
        xb = F.relu(self.maxpool2d3(xb))
        
        xb = F.relu(self.conv4(xb))
        xb = F.relu(self.batchnorm4(xb))
        
        xb = F.relu(self.conv5(xb))
        xb = F.relu(self.batchnorm5(xb))
        
        xb = F.relu(self.conv6(xb))
        xb = F.relu(self.maxpool2d6(xb))
        
        xb = F.relu(self.flatten(xb))
        
        xb = F.relu(self.fc1(xb))
        xb = F.relu(self.dropout1(xb))
        
        xb = F.relu(self.fc2(xb))
        xb = F.relu(self.dropout2(xb))
        
        xb = F.relu(self.fc3(xb))
        
        return xb.view(-1, xb.size(1))

In [5]:
#Capas preentrenadas
import torchvision
from torchvision import models


model_VGG16 = torchvision.models.vgg16(pretrained=True, progress=True)
pesos_capa1 = model_VGG16.features[0].weight
pesos_capa2 = model_VGG16.features[2].weight
pesos_capa1.requires_grad = False
pesos_capa2.requires_grad = False

In [13]:
class VGG16DWSep(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1layer1=nn.Conv2d(1,64,kernel_size=3,padding=1)
        self.conv1layer1.weight = pesos_capa1
        self.conv1layer2=nn.Conv2d(64,64,kernel_size=3,padding=1)
        self.conv1layer2.weight = pesos_capa2
        self.maxpool2d1=nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv2layer1=DWSepConv2d(64,128,3,1)
        self.conv2layer2=DWSepConv2d(128,128,3,1)
        self.maxpool2d2=nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv1=DWSepConv2d(128,256,3,1)
        self.batchnorm1=nn.BatchNorm2d(256)
        
        self.conv2=DWSepConv2d(256,256,3,1)
        self.batchnorm2=nn.BatchNorm2d(256)
        
        self.conv3=DWSepConv2d(256,512,3,1)
        self.maxpool2d3=nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.conv4=DWSepConv2d(512,512,3,1)
        self.batchnorm4=nn.BatchNorm2d(512)
        
        self.conv5=DWSepConv2d(512,512,3,1)
        self.batchnorm5=nn.BatchNorm2d(512)
        
        self.conv6=DWSepConv2d(512,512,3,1)
        self.maxpool2d6=nn.MaxPool2d(kernel_size=2,stride=2)
        
        self.flatten=nn.Flatten()
        
        self.fc1 = nn.Linear(256, 1024)
        self.dropout1=nn.Dropout(.7)
        
        self.fc2 = nn.Linear(1024, 512)
        self.dropout2=nn.Dropout(.5)
        
        self.fc3 = nn.Linear(512, 2)
    
       
    def forward():
        xb = xb.view(-1, 3, 224, 224)
        xb = F.relu(self.conv1layer1(xb.float()))
        xb = F.relu(self.conv1layer2(xb))
        xb = F.relu(self.maxpool2d1(xb))
        
        xb = F.relu(self.conv2layer1(xb))
        xb = F.relu(self.conv2layer2(xb))
        xb = F.relu(self.maxpool2d2(xb))
        
        xb = F.relu(self.conv1(xb))
        xb = F.relu(self.batchnorm1(xb))
        
        xb = F.relu(self.conv2(xb))
        xb = F.relu(self.batchnorm2(xb))
        
        xb = F.relu(self.conv3(xb))
        xb = F.relu(self.maxpool2d3(xb))
        
        xb = F.relu(self.conv4(xb))
        xb = F.relu(self.batchnorm4(xb))
        
        xb = F.relu(self.conv5(xb))
        xb = F.relu(self.batchnorm5(xb))
        
        xb = F.relu(self.conv6(xb))
        xb = F.relu(self.maxpool2d6(xb))
        
        xb = F.relu(self.flatten(xb))
        
        xb = F.relu(self.fc1(xb))
        xb = F.relu(self.dropout1(xb))
        
        xb = F.relu(self.fc2(xb))
        xb = F.relu(self.dropout2(xb))
        
        xb = F.relu(self.fc3(xb))
        
        return xb.view(-1, xb.size(1))

In [28]:
class EarlyStopping():
    
    def __init__(self, modo='min', paciencia=5, porcentaje=False, tol=0):
        self.modo = modo
        self.wait = paciencia
        self.perc = porcentaje
        self.tol = tol
        self.best = None
        self.bestep = 0
    
    def mejor(self, metrica_validacion):
        
        '''
        Retorna True cuando se mejora el mejor valor obtenido en métrica. 
        En caso contrario retorna False.
        '''
        
        if self.best == None:
            self.best = metrica_validacion
            return True
        if self.perc == False:
            if self.modo == 'min':
                if metrica_validacion < self.best - self.tol:
                    self.best = metrica_validacion
                    self.bestep = 0
                    return True
                else:
                    self.bestep += 1
                    return False
            else:
                if metrica_validacion > self.best + self.tol:
                    self.best = metrica_validacion
                    self.bestep = 0
                    return True
                else:
                    self.bestep += 1
                    return False
        else: 
            if self.modo == 'min':
                if metrica_validacion < (1-self.tol)*self.best:
                    self.best = metrica_validacion
                    self.bestep = 0
                    return True
                else:
                    self.bestep += 1
                    return False
            else:
                if metrica_validacion > (1+self.tol)*self.best:
                    self.best = metrica_validacion
                    self.bestep = 0
                    return True
                else:
                    self.bestep += 1
                    return False
        
    
    def deberia_parar(self, metrica_validacion):
        comp = self.mejor(metrica_validacion)
        if not comp and self.bestep == self.wait:
            return True
        else:
            return False
                 

In [29]:
torch.cuda.is_available()

True

In [30]:
#Ciclo de entrenamiento
from torch import optim

tl = []
losses = []
model = VGG16DWSep()
opt = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
loss_func = F.cross_entropy
num_epoch = 20
es  = EarlyStopping()
for epoch in range(num_epoch):
    #ciclo de entrenamiento
    for i, (images, labels) in enumerate(tl):
        
        outputs = model(images)
        loss = loss_func(outputs, labels)
        losses.append(loss.item())
        
        # Backprop and perform Adam optimisation
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    #ciclo de validación
    metrica_validacion = 0
    if es.deberia_parar(metrica_validacion):
        break
    

In [ ]:
#P3